In [1]:
from premise import *
import bw2data, bw2io, bw2calc
#bw2data.projects.set_current("bw25_ei310")
bw2data.projects.set_current("ecoinvent-3.12-cutoff")

/opt/homebrew/Caskroom/miniforge/base/envs/premise/lib/python3.11/site-packages/scikits/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__('pkg_resources').declare_namespace(__name__)


19:38:36+0100 [warning  ] Can't import `SimaProBlockCSVImporter` - please install `bw2io` with `pip install bw2io[multifunctional]` or install `multifunctional` and `bw_simapro_csv` manually.


In [2]:
#clear_inventory_cache()

In [3]:
scenarios = [
    #{"model": "remind", "pathway": "SSP1-NDC", "year": 2025,},
    #{"model": "remind", "pathway": "SSP1-NPi", "year": 2025,},
    #{"model": "remind", "pathway": "SSP1-PkBudg650", "year": 2025,},
    #{"model": "remind", "pathway": "SSP1-PkBudg1000", "year": 2025,},
    #{"model": "remind", "pathway": "SSP2-rollBack", "year": 2025,},
    #{"model": "remind", "pathway": "SSP2-NDC", "year": 2025,},
    #{"model": "remind", "pathway": "SSP2-NPi", "year": 2050,},
    #{"model": "remind", "pathway": "SSP2-PkBudg650", "year": 2025,},
    #{"model": "remind", "pathway": "SSP2-PkBudg1000", "year": 2025,},
    #{"model": "remind", "pathway": "SSP3-rollBack", "year": 2025,},
    #{"model": "remind", "pathway": "SSP3-NDC", "year": 2025,},
    #{"model": "remind", "pathway": "SSP3-NPi", "year": 2025,},
    #{"model": "remind-eu", "pathway": "SSP2-NDC", "year": 2050,},
    #{"model": "remind", "pathway": "SSP3-PkBudg1000", "year": 2025,},
    #{"model": "remind-eu", "pathway": "SSP2-NPi", "year": 2025,},
    #{"model": "remind-eu", "pathway": "SSP2-PkBudg650", "year": 2050,},
    #{"model": "remind-eu", "pathway": "SSP2-PkBudg1000", "year": 2025,},

    #{"model": "image", "pathway": "SSP1-L", "year": 2025,},
    #{"model": "image", "pathway": "SSP1-Ma", "year": 2025,},
    #{"model": "image", "pathway": "SSP1-VLLO", "year": 2025,},
    #{"model": "image", "pathway": "SSP2-L", "year": 2025,},
    #{"model": "image", "pathway": "SSP2-M", "year": 2025,},
    #{"model": "image", "pathway": "SSP2-VLHO", "year": 2025,},
    #{"model": "image", "pathway": "SSP5-H", "year": 2025,},
    #{"model": "image", "pathway": "SSP3-H", "year": 2025,},

    #{"model": "tiam-ucl", "pathway": "SSP2-Base", "year": 2050,},
    #{"model": "tiam-ucl", "pathway": "SSP2-RCP19", "year": 2025,},
    #{"model": "tiam-ucl", "pathway": "SSP2-RCP26", "year": 2025,},
    #{"model": "tiam-ucl", "pathway": "SSP2-RCP45", "year": 2025,},

    #{"model": "remind", "pathway": "SSP2-NPi", "year": 2030,},
    #{"model": "remind", "pathway": "SSP2-NPi", "year": 2035,},
    #{"model": "remind", "pathway": "SSP2-NPi", "year": 2050,},
    #{"model": "remind", "pathway": "SSP2-NPi", "year": 2040,},
    #{"model": "remind", "pathway": "SSP2-NPi", "year": 2045,},
    

    #{"model": "remind", "pathway": "SSP2-NDC", "year": 2030,},
    #{"model": "remind", "pathway": "SSP2-NDC", "year": 2035,},
    #{"model": "remind", "pathway": "SSP2-NDC", "year": 2040,},
    #{"model": "remind", "pathway": "SSP2-NDC", "year": 2045,},
    #{"model": "remind", "pathway": "SSP2-NDC", "year": 2050,},

    {"model": "remind", "pathway": "SSP2-PkBudg650", "year": 2035,},
    #{"model": "remind", "pathway": "SSP2-PkBudg650", "year": 2050,},

    #{"model": "remind", "pathway": "SSP2-rollBack", "year": 2030,},
    #{"model": "remind", "pathway": "SSP2-rollBack", "year": 2035,},
    #{"model": "remind", "pathway": "SSP2-rollBack", "year": 2040,},
    #{"model": "remind", "pathway": "SSP2-rollBack", "year": 2045,},
    #{"model": "remind", "pathway": "SSP2-rollBack", "year": 2050,},

    #{"model": "gcam", "pathway": "SSP2-Base", "year": 2050,},
    #{"model": "gcam", "pathway": "SSP2-RCP26", "year": 2050,},
]


ndb = NewDatabase(
        scenarios = scenarios,
        source_db="ecoinvent-3.12-cutoff",
        source_version="3.12",
        key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
        biosphere_name="biosphere3",
        use_absolute_efficiency=True,
        system_model="cutoff"
)


premise v.(2, 3, 3)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your Brightway project:                                   |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

In [4]:
ndb.update()

Processing scenarios for all sectors: 100%|█| 1/1 [01:34<00:00, 94.80s

Done!



In [5]:
ndb.write_db_to_brightway()

Write new database(s) to Brightway.
Running all checks...
Minor anomalies found: check the change report.
19:41:18+0100 [warning  ] Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|████████████████████████████████████| 38796/38796 [01:00<00:00, 636.83it/s]


19:42:20+0100 [info     ] Vacuuming database            
Created database: ei_cutoff_3.12_remind_SSP2-PkBudg650_2035 2025-12-03
Generate scenario report.
Report saved under /Users/romain/GitHub/premise/dev/export/scenario_report.
Generate change report.
Report saved under /Users/romain/GitHub/premise/dev/export/change reports/.


In [5]:
ndb.generate_change_report()

Generate change report.
Report saved under /Users/romain/GitHub/premise/dev/export/change reports/.


In [3]:
from premise.activity_maps import mapping_to_dataframe

In [2]:
mapping_to_dataframe?

Signature:
mapping_to_dataframe(
    scenario: 'Dict[str, Union[str, List[dict]]]',
    original_database: 'Optional[List[dict]]' = None,
) -> 'pd.DataFrame'
Docstring:
Convert mapping dictionaries into a grouped :class:`pandas.DataFrame`.

:param scenario: Scenario dictionary containing a ``database`` key with the ecoinvent database.
:type scenario: Dict[str, Union[str, List[dict]]]
:param original_database: Optional original database used to reload the scenario.
:type original_database: Optional[List[dict]]
:return: DataFrame with aggregated mapping information and location lists.
:rtype: pandas.DataFrame
File:      ~/GitHub/premise/premise/activity_maps.py
Type:      function

In [4]:
df = mapping_to_dataframe(
    ndb.scenarios[0],
    ndb.database
)

In [5]:
df

,Sector,Category,Name,Reference product,Location
0,biomass,biomass - residual,supply of forest residue,forest residue,"RER, US"
1,,biomass wood - purpose grown,"flax shive to generic market for wood chips, d...","wood chips, dry, measured as dry mass",RER
2,,,"hemp hurd to generic market for wood chips, dr...","wood chips, dry, measured as dry mass",FR
3,,,hemp residues to generic market for wood chips...,"wood chips, dry, measured as dry mass",FR
4,,,"market for wood chips, dry, measured as dry mass","wood chips, dry, measured as dry mass","RER, RoW"
...,...,...,...,...,...
3586,,"moped, gasoline","transport, Scooter, gasoline, 4-11kW, EURO-5","transport, Scooter, gasoline, 4-11kW, EURO-5",CH
3587,,"motorcycle, 50-250cc, battery electric","transport, Motorbike, battery electric, 11-35kW","transport, Motorbike, battery electric, 11-35kW",CH
3588,,"motorcycle, 50-250cc, gasoline","transport, Motorbike, gasoline, 11-35kW, EURO-5","transport, Motorbike, gasoline, 11-35kW, EURO-5",CH
3589,,"motorcycle, >250cc, battery electric","transport, Motorbike, battery electric, >35kW","transport, Motorbike, battery electric, >35kW",CH


In [6]:
df.to_excel("mapping_ei312.xlsx")

In [3]:
ndb.update()

Generate scenario report.
Report saved under /Users/romain/GitHub/premise/dev/export/scenario_report.


In [7]:
db = ndb.scenarios[0]["database"]

Write new database(s) to Brightway.
Running all checks...
Minor anomalies found: check the change report.


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:36


Title: Writing activities to SQLite3 database:
  Started: 10/22/2025 10:23:46
  Finished: 10/22/2025 10:24:23
  Total time elapsed: 00:00:36
  CPU %: 88.40
  Memory %: 13.23
Created database: ei_cutoff_3.11_gcam_SSP2-Base_2050 2025-10-22
Running all checks...
Minor anomalies found: check the change report.


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:37


Title: Writing activities to SQLite3 database:
  Started: 10/22/2025 10:39:06
  Finished: 10/22/2025 10:39:44
  Total time elapsed: 00:00:37
  CPU %: 87.60
  Memory %: 13.53
Created database: ei_cutoff_3.11_gcam_SSP2-RCP26_2050 2025-10-22
Generate scenario report.
Report saved under /Users/romain/GitHub/premise/dev/export/scenario_report.
Generate change report.
Report saved under /Users/romain/GitHub/premise/dev/export/change reports/.


In [6]:
ei = bw2data.Database("ecoinvent-3.11-cutoff")

market for steel, unalloyed SSA 126.5147048
market for steel, unalloyed NEN 3.9033624000000002
market for steel, unalloyed DEU 30.9445126
market for steel, unalloyed CHA 627.594136
market for steel, unalloyed NES 33.7435476
market for steel, unalloyed ESC 28.518298299999998
market for steel, unalloyed USA 125.8700527
market for steel, unalloyed ECS 3.1706321
market for steel, unalloyed FRA 12.4022449
market for steel, unalloyed CAZ 20.2429662
market for steel, unalloyed IND 245.47239739999998
market for steel, unalloyed REF 44.1181361
market for steel, unalloyed JPN 77.40011770000001
market for steel, unalloyed ESW 16.153458699999998
market for steel, unalloyed MEA 120.0108343
market for steel, unalloyed ECE 12.3216091
market for steel, unalloyed UKI 7.385962899999999
market for steel, unalloyed EWN 25.3922545
market for steel, unalloyed LAM 101.8951347
market for steel, unalloyed ENC 12.5858469
market for steel, unalloyed OAS 168.2978058
market for steel, unalloyed GLO 0.0
steel produ

In [7]:
bw2data.projects.set_current("ecoinvent-3.12-cutoff")

In [8]:
bw2data.databases

Databases dictionary with 2 object(s):
	biosphere3
	ecoinvent-3.12-cutoff

In [9]:
bio = bw2data.Database("biosphere3")